# Db2 Data Management Console Workload Generation Helper Classes

This code includes all of the helper classes used to run scripts of SQL statements

### Imports

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### Db2 Class

In [14]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, userid, password, profile=""):
        credentials = {'userid':userid, 'password':password}
        r = requests.post(self.url+'/auth/tokens', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = r.json()['token']
            if profile == "":
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken}
                return True
            else:
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken, 'X-DB-Profile': profile}
                return True
        else:
            print ('Unable to authenticate, no bearer token obtained')
            return False
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def getSchemas(self):
        return self.getRequest('/schemas')
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getCurrentApplicationsConnections(self, includeSystem='true'):
        return self.getRequest('/metrics/applications/connections/current/list?&include_sys='+str(includeSystem))
    
    def getInflightCount(self, startTime, endTime):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?start='+str(startTime)+'&end='+str(endTime));
       
    def getInflightCurrentList(self, includeSystem='true'):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?'+'&include_sys='+str(includeSystem));
    
    def getIndividualStatementExecution(self, startTime, endTime, limit=100, includeSystem='false'):
        return self.getRequest('/metrics/statements/evmon_activity?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem)+'&offset=0&limit='+str(limit))

    def getFiles(self, path):
        return self.getRequest('/home'+path)
    
    def getUsers(self):
        return self.getRequest('/users')
    
    def getTablesMetrics(self, startTime, endTime, includeSystem='false'):
        return self.getRequest('/metrics/tables?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem));

    def getAverageResponseTime(self, startTime, endTime):
        return self.getRequest('/metrics/average_response_time?start='+str(startTime)+'&end='+str(endTime));    
    
    def getUnitsOfWork(self, startTime, endTime):
        return self.getRequest('/applications/uow?start='+str(startTime)+'&end='+str(endTime));    
    
    def getSchemaSize(self, startTime, endTime, tabSchema):
        return self.getRequest('/metrics/storage/schemas/'+tabSchema+'/timeseries?start='+str(startTime)+'&end='+str(endTime));
  
    def getSearchViewList(self, searchtext, show_systems="false"):
        return self.getRequest('/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchTableList(self, searchtext):
        return self.getRequest('/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');
              
    def getRowsRead(self, startTime, endTime):
        return self.getRequest('/metrics/rows_read?start='+str(startTime)+'&end='+str(endTime));

    def getResponseTime(self, startTime, endTime):
        return self.getRequest('/metrics/response_time?start='+str(startTime)+'&end='+str(endTime));

    def getStatementsCount(self, startTime, endTime):
        return self.getRequest('/metrics/statements_count?start='+str(startTime)+'&end='+str(endTime));

    def getPackageCacheStatement(self, startTime, endTime, is_average='true'):
        show_systems='false'
        return self.getRequest('/metrics/statements/package_cache?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(show_systems)+'&is_average='+str(is_average))
    
    def postSearchObjects(self, obj_type, search_text, rows_return=100, show_systems='false', is_ascend='true'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/admin/'+str(obj_type)+'s',json);
                
    def putFile(self, filename, path):
        with open(filename, 'rb') as f:
            r = requests.post(self.url+'/home_content/path', files={filename: f}, verify = self.verify, headers=self.headers, proxies = self.proxies)
            
    def getTablesInSchema(self, schema):
        return self.getRequest('/schemas/'+str(schema)+'/tables');
    
    def getTableStorageBySchema(self):
        return self.getRequest('/metrics/storage/schemas?end=0&include_sys=true&limit=1000&offset=0&start=0')
    
    def getCurrentPackageCacheList(self, show_systems="false"):
        return self.getRequest('/metrics/statements/package_cache/current/list?include_sys='+str(show_systems))
    
    def getProfile(self,profile):
        return self.getRequest('/dbprofiles/'+profile)    
    
    def getMonitorStatus(self):
        return self.getRequest('/monitor')    
    
    def runSQLScript(self, profile, user, password, sqlText, rowLimit=10):
        if self.authenticate(user, password, profile) :
            json = self.getJSON(self.runSQL(sqlText, rowLimit))
            if 'id' in json :
                runID = json['id'] 
            else :
                print('Operational Credentials to Connect to Database Not Available in the Db2 Console Repository')
                return json
            json = self.getJSON(self.getSQLJobResult(runID))
            if 'errors' in json :
                return json
            fulljson = json

            while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
                json = self.getJSON(self.getSQLJobResult(runID))
                for results in json['results'] :
                    fulljson['results'].append(results)
                time.sleep(0.25) 
            return fulljson
        else :
            print('Could not authenticate')
            
    def displayScriptResults(self, json):
        for results in json['results']:
            print('Statement: '+str(results['index'])+': '+results['command'])
            print('Runtime ms: '+str(results['runtime_seconds']*1000))
            if 'error' in results : 
                print(results['error'])
            elif 'rows' in results :
                df = pd.DataFrame(results['rows'],columns=results['columns'])
                display(df)
            else :
                print('No errors. Row Affected: '+str(results['rows_affected']))
            print()
        
    def appendScriptResults(self, df, profile, json):    
        error = ''
        rows = 0
        if 'error' in json :
            print('SQL Service Failed')
        else :
            for results in json['results']:
                if 'error' in results : 
                    error = results['error']
                if 'rows_affected' in results : 
                    rows = results['rows_affected']
                df = df.append({'profile':profile,'index':results['index'], 'statement':results['command'], 'error':error, 'rows_affected': rows, 'runtime_ms':(results['runtime_seconds']*1000)}, ignore_index=True)
        return df
    
    def batchScript(self, profileList, scriptList, user, password, profileReps, scriptReps) :
        df = pd.DataFrame(columns=['profile', 'index', 'statement', 'error', 'rows_affected', 'runtime_ms'])
        for x in range(0, profileReps):
            print("Running repetition: "+str(x))
            for profile in profileList :
                print("  Running scripts against: "+profile)
                for y in range(0, scriptReps) :
                    print("    Running script repetition: "+str(y))
                    for script in scriptList :
                        json = self.runSQLScript(profile, user, password, script)
                        while 'errors' in json:
                            print('    * Trying again *')
                            json = self.runSQLScript(profile, user, password, script)
                        df = self.appendScriptResults(df, profile, json)
        return df

    def getCurrentPackageCacheListDF(self, profile, user, password, topX=10) :
        self.authenticate(user, password, profile)
        r = self.getCurrentPackageCacheList("false")
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            if json['count'] > 0:  
                df = pd.DataFrame(json_normalize(json['resources']))
                df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
                if df.empty == True :
                    print("No statements in the package cache")
                else :
                    display(df[['stmt_text','stmt_exec_time_ms','stmtid']].head(topX))
            else: 
                print('No data returned') 
                return pd.DataFrame()
        else:
            print(self.getStatusCode(r))

    def getPackageCacheStatementHistory(self, profile, user, password,startTime, endTime, is_average='true', topX=10):
        self.authenticate(user, password, profile)
        r = self.getPackageCacheStatement(startTime, endTime, is_average)
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            if json['count'] > 0:  
                df = pd.DataFrame(json_normalize(json['resources']))
                df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
                if df.empty == True :
                    print("No statements in the package cache")
                else :
                    display(df[['stmt_text','stmt_exec_time_ms','stmtid']].head(topX))
            else: 
                print('No data returned') 
                return pd.DataFrame()
        else:
            print(self.getStatusCode(r))


### Conversion Classes

In [6]:
# Setup data frame set calculation functions
def epochtotimeseries(epoch):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch/1000))
def KBtoGB(kb):
    return kb/1024/1024

### Time Series Calculations

In [7]:
import time
from datetime import date
endTime = int(time.time())*1000
startTime = endTime-(600*1000)
oneWeek = 604800000
oneDay = oneWeek / 7
oneHour = oneDay / 24

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]